In [ ]:
import os
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler, LabelBinarizer
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

import tensorflow as tf

from PIL import Image

import matplotlib.pyplot as plt

In [ ]:
!pip show mlxtend

In [ ]:
# List of emotions on the dataset we want to include (using comments -#- to exlcude).

EMOTIONS=[
    #'angry',
    #'disgust',
    #'fear',
    'happy',
    'neutral',
    'sad',
    'surprise',
]

In [ ]:
# Load dataset images into the DataFrame.

dataset = []

for emotion in EMOTIONS:
    if os.path.isdir(f'dataset/{emotion}'):
        for image_filename in os.listdir(f'dataset/{emotion}'):
            image = Image.open(f'dataset/{emotion}/{image_filename}')
            pixels = np.array(image).astype(float)
            dataset.append((pixels, emotion))

df = pd.DataFrame(dataset, columns=['pixels', 'emotion'])

In [ ]:
# Normalizacion of the data.

X = np.array([MinMaxScaler().fit_transform(e) for e in df['pixels']])
lb = LabelBinarizer()
Y = lb.fit_transform(df['emotion'])

In [ ]:
# Load trained model.

MODEL='models/modelConv.h5'

model = tf.keras.models.load_model(MODEL)

In [ ]:
# Show confusion matrix (accuracy by emotion).

Y_pred = model.predict(X)

Y_labels = lb.inverse_transform(Y)
Y_pred_labels = lb.inverse_transform(Y_pred)

m = confusion_matrix(Y_labels, Y_pred_labels)

plot_confusion_matrix(m, class_names=EMOTIONS, show_absolute=False, show_normed=True)
plt.show()

In [ ]:
pred_fail_idx = np.where(np.not_equal(Y_labels, Y_pred_labels))

In [ ]:
SHOW=[5,5]

fig, ax = plt.subplots(SHOW[0], SHOW[1], sharex=True, sharey=True, figsize=(20,20))


for i in range(SHOW[0]):
    for j in range(SHOW[1]):
        idx = pred_fail_idx[0][i*SHOW[0]+j]
        ax[i][j].imshow(X[idx], cmap='gray')
        ax[i][j].set_xticks([])
        ax[i][j].set_yticks([])   
        ax[i][j].set_title(f'Dataset: {Y_labels[idx]} || Model: {Y_pred_labels[idx]}')
    
plt.show()